In [16]:
import pandas as pd
df = pd.read_csv('amazon_cells_labelled1.txt', names=['sentence', 'label'], sep='\t')

In [17]:
df

,sentence,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
1996,The screen does get smudged easily because it ...,0
1997,What a piece of junk.. I lose more calls on th...,0
1998,Item Does Not Match Picture.,0
1999,The only thing that disappoint me is the infra...,0


In [18]:
from sklearn.model_selection import train_test_split
kalimat = df['sentence'].values
y = df['label'].values
kalimat_latih, kalimat_test, y_latih, y_test = train_test_split(kalimat, y, test_size=0.2)

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=250, oov_token='x')
tokenizer.fit_on_texts(kalimat_latih) 
tokenizer.fit_on_texts(kalimat_test)
 
sekuens_latih = tokenizer.texts_to_sequences(kalimat_latih)
sekuens_test = tokenizer.texts_to_sequences(kalimat_test)
 
padded_latih = pad_sequences(sekuens_latih, maxlen=20) 
padded_test = pad_sequences(sekuens_test, maxlen=20)

In [20]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(250, 16, input_length=20),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer=tf.optimizers.Adam(), metrics=['accuracy'])

In [21]:
accuracy_threshold = 90e-2
class my_callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        if logs.get('val_accuracy') > accuracy_threshold:
            print('Reached 90%, stop epoch')
            self.model.stop_training = True

In [22]:
num_epochs = 30
history = model.fit(padded_latih, y_latih, epochs=num_epochs, 
                    validation_data=(padded_test, y_test), verbose=2, callbacks = [my_callbacks()])

Epoch 1/30
50/50 - 3s - loss: 0.6911 - accuracy: 0.5400 - val_loss: 0.6837 - val_accuracy: 0.6584 - 3s/epoch - 53ms/step
Epoch 2/30
50/50 - 0s - loss: 0.6261 - accuracy: 0.6975 - val_loss: 0.4952 - val_accuracy: 0.7781 - 337ms/epoch - 7ms/step
Epoch 3/30
50/50 - 0s - loss: 0.4484 - accuracy: 0.7994 - val_loss: 0.4021 - val_accuracy: 0.8105 - 288ms/epoch - 6ms/step
Epoch 4/30
50/50 - 0s - loss: 0.3502 - accuracy: 0.8494 - val_loss: 0.3453 - val_accuracy: 0.8429 - 274ms/epoch - 5ms/step
Epoch 5/30
50/50 - 0s - loss: 0.3041 - accuracy: 0.8625 - val_loss: 0.3382 - val_accuracy: 0.8454 - 271ms/epoch - 5ms/step
Epoch 6/30
50/50 - 0s - loss: 0.2822 - accuracy: 0.8756 - val_loss: 0.4681 - val_accuracy: 0.7880 - 236ms/epoch - 5ms/step
Epoch 7/30
50/50 - 0s - loss: 0.2925 - accuracy: 0.8756 - val_loss: 0.3297 - val_accuracy: 0.8579 - 230ms/epoch - 5ms/step
Epoch 8/30
50/50 - 0s - loss: 0.2513 - accuracy: 0.8956 - val_loss: 0.3104 - val_accuracy: 0.8753 - 239ms/epoch - 5ms/step
Epoch 9/30
50/50 -